# import modules

In [242]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# from sklearn.pipeline import Pipeline
import datetime
import time

# Data set 불러오기

In [243]:
# 여러분의 폴더가 저장되어있는 위치 경로를 써 주세요
# 그리고 데이터파일(주최측에서 제공한) 의 이름을 Test set 으로 변셩해 주세요
# 한인욱 directory = 'C:/Users/goran/Desktop/Python/Project/20S_Bigcon/Test'
# 김정현 directory = 'C:/Users/Coby/Desktop/20S_Bigcon/20S_Bigcon/Test'
# 신예진 directory = 'C:/Users/Rstudio/Desktop/Bigcon2020/20S_Bigcon/Test'
# 이규민 directory = 'C:/Statistics/20S_Bigcon/Test'
# 최정욱 directory = 'C:/Users/lg/Desktop/Bigcon2020/Test'
directory = 'C:/Statistics/20S_Bigcon/Test'

In [244]:
# 그러면 이렇게 그 폴더 안의 Train set.xlsx 을 잘 읽어옵니다! 
X_df=pd.read_excel(directory+'/Test set.xlsx',header = 1) 

In [245]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [246]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN


In [247]:
X.drop(columns='취급액',inplace= True)

# 결측치 처리 및 변수늘리기

In [248]:
X.isnull().sum()

방송일시        0
노출(분)    1111
마더코드        0
상품코드        0
상품명         0
상품군         0
판매단가        0
dtype: int64

## 노출(분) 결측치 처리

노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다. <br>
아마 같은 시간에 방송되어진 값을 평균해서 채우면 될듯?

In [249]:
X.loc[:,'평균방송분'] = X.loc[:,'노출(분)']

In [250]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대채한 후에 평균떄려도 아무 상관 없다는 이야기!
(X.loc[:,'노출(분)']==0).sum()

0

In [251]:
# 노출(분) 을 제일 먼저 시작한 시간의 값으로 만드는 FUNCTION 정의
# 아직 맞는지는 잘 몰라서 안맞는 경우 수정해주세요.
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    df['노출(분)'] = df['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] # / num 을 붙여주면 평균으로 만들게 됩니다.
    return df

In [252]:
X = NA_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000


## 얼마나 같은 방송일시에 겹쳐있는지 변수늘리기

In [253]:
X.loc[:,'겹치는수'] = 1

In [254]:
for idx in range(1,len(X))  :
    if ~(X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']): # 앞과 마더코드가 다를때!
        continue # 아무 관련 없으므로 다음 for 문으로 넘어가도록 합시다
    elif (X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']) : # 앞 구간과 같아버리면 같은게 지속되고있는거겟죠!
        if X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']: # 만약 전의 방송일시와 지금이 일치하면 그냥 전 index 의 겹치는 수를 씁니다. 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']
        elif ~(X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']): # 만약 전의 방송일시와 지금이 다르다면 , 마더코드는 같은데 방송일시가 다르단거겟죠? index 를 증가시킵니다.
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']+1

In [255]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1


## 평균 방송분 변수늘리기

In [256]:
def MEAN_minute(df=X):
    # 곁측치를 0으로 대체
    df['평균방송분'] = df['평균방송분'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'평균방송분'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'평균방송분'] = df.loc[start,'평균방송분'] / num
    return df

In [257]:
X = MEAN_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2
...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,1
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,2
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,20.000000,3
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1


## 상품군 결측치 처리

 판매단가가 0 인 상품은 예측에서 제외

In [258]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [259]:
X = NA_goods(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2
...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1


##  토요일 곁측치 처리

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [260]:
def NA_Saturday(df = X):
    t = ['월','화','수','목','금','토','일']
    r = datetime.datetime.today()
    def printDayOfTheWeek(year, month, day):
        dayOfTheWeek = ['월','화','수','목','금','토','일']
        return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]
    day_list = []
    for date in X.iloc[:,0]:
        date = str(date)
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        first = printDayOfTheWeek(year,month,day)
        second = date[11:16]
        day_list.append(first+second)
    X["요일/시간"] = day_list
#    for i in range(len(df)):
#        time = df.loc[i,'요일/시간'] # 여기도 요일/시간 인 loc 로 바꾸었습니다!
#        if (time[0:3]=="토18" and int(time[4:])<20):
#            df = df.drop(i)
    return df

In [261]:
NA_Saturday(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40
...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20


In [262]:
#check 하는 용도라 뺴놨습니다!
for i in range(len(NA_Saturday(X))):
    time = X.loc[i,'요일/시간'] # 여기 loc 로 바꾸었습니다. 왜냐하면 앞에서 평균방송분 열을 추가해야되서 열이 하나 늘어났어요
    if time[0:3]=="토18":
        print(time)

토18:20
토18:40
토18:20
토18:20
토18:40
토18:40
토18:00
토18:40


In [263]:
X = NA_Saturday(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40
...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20


힘들게 코드를 짰으나 토요일 18:00~18:20에 해당하는 방송은 없었다.............................

# 변수 늘리기

## 시간 데이터 늘리기

In [264]:
# 시각은 우선 시,분을 합쳐서 하나의 값으로 만들었습니다.
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df['시분'] = df['시'] + ':' + df['분']
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [265]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,2020-06-01,2020,06,01,6.333333,06:20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,2020-06-01,2020,06,01,6.666667,06:40
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,2020-06-01,2020,06,01,7.000000,07:00
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,2020-06-01,2020,06,01,7.333333,07:20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,2020-06-01,2020,06,01,7.666667,07:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20,2020-07-01,2020,07,01,1.333333,01:20


## 시청률 평균 적용

주파수를 사용하지 않는 케이블, 위성방송, IPTV의 등장에 따라 셋톱박스에서 조사가 가정해져서 시청률이 좀 더 정확해졌으며, 시청률 조사지역은 점차 확대되어 수도권, 5대 광역시, 중소도시등이 포함되어 2019년에는 모든 케이블 방송권역에서 조사가 이루어지고 있다. 2020년 현재 TNMS는 전국(제주도 및 읍·면 지역 포함) 4,000여 가구에 거주하는 약 9천명을 셋탑박스를 조사(TMNS 홈페이지 기준), 닐슨 코리아에서는 피플미터기로 4,220가구(닐슨 컴퍼니 코리아 홈페이지 기준)를 조사한다.

+) http://www.mediaus.co.kr/news/articleView.html?idxno=22809

In [266]:
# 한인욱 directory2 = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'
# 김정현 directory2 = 'C:/Users/Coby/Desktop/20S_Bigcon/20S_Bigcon'
# 신예진 directory2 = 'C:/Users/Rstudio/Desktop/Bigcon2020/20S_Bigcon'
# 이규민 directory2 = 'C:/Statistics/20S_Bigcon'
# 최정욱 directory2 = 'C:/Users/lg/Desktop/Bigcon2020'
directory2 = 'C:/Statistics/20S_Bigcon'
weekend_df=pd.read_csv(directory2+'/weekend.csv')
weekday_df=pd.read_csv(directory2+'/weekday.csv')
#weekend_df=pd.read_excel(directory2+'/weekend.xlsx')
#weekday_df=pd.read_excel(directory2+'/weekday.xlsx')

In [267]:
weekend=weekend_df.copy()
weekday=weekday_df.copy()
weekend.index = weekend['시각']
weekday.index = weekday['시각']

weekday

,시각,시간,시청률 평균,시각.1
시각,,,,
600,600,06:00,0.001407,600
601,601,06:01,0.001648,601
602,602,06:02,0.001492,602
603,603,06:03,0.001658,603
604,604,06:04,0.001849,604
...,...,...,...,...
2955,2955,05:55,0.001136,2955
2956,2956,05:56,0.001231,2956
2957,2957,05:57,0.000950,2957


In [268]:
weekend

,시각,시간,시청률 평균,시각.1
시각,,,,
600,600,06:00,0.002175,600
601,601,06:01,0.002434,601
602,602,06:02,0.002747,602
603,603,06:03,0.003976,603
604,604,06:04,0.003795,604
...,...,...,...,...
2955,2955,05:55,0.002018,2955
2956,2956,05:56,0.002819,2956
2957,2957,05:57,0.002608,2957


In [269]:
time_list=['0605','0606','0607','0612','0613','0614','0619','0620','0621','0626','0627','0628']

In [270]:
X['월일'] = X['월'].astype(str) + X['일'].astype(str)
X.loc[:,'시분'] = X.loc[:,'시분'].astype(str)
X['시간열']=X['시분'].str.split(':').str[0].astype(str) + X['시분'].str.split(':').str[1].astype(str)
X['시간열']=X['시간열'].astype(int)

In [271]:
for i in range(len(X)):
    if X.loc[i,'시간열'] < 600:
         X.loc[i,'시간열']  += 2400

In [272]:
X['정수노출(분)']=X['노출(분)'].round(0).astype(int)

In [273]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분)
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,2020-06-01,2020,06,01,6.333333,06:20,0601,620,20
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,2020-06-01,2020,06,01,6.666667,06:40,0601,640,20
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,2020-06-01,2020,06,01,7.000000,07:00,0601,700,20
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,2020-06-01,2020,06,01,7.333333,07:20,0601,720,20
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,2020-06-01,2020,06,01,7.666667,07:40,0601,740,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20,2020-07-01,2020,07,01,1.333333,01:20,0701,2520,20


In [274]:
X['시청률평균'] = 0

In [275]:
for idx in range(len(X)-1):
    if X.loc[idx,'월일'] in time_list:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: # 앗 이경우에는 뒤의값과 같네요. 지속입니다
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : # 앗 이경우에는 전값이 더 크네요? 그 경우에는 날짜가 바뀐것입니다.
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
        # 이 경우에는 얼마만큼 노출되었는지를 고려해서 그만큼을 MEAN 을 시켜줍니다. 
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : # 뒤의 값이 더 크네요? 그 경우에는 상품이 바뀐것이죠.
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
    else:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: # 앗 이경우에는 뒤의값과 같네요. 지속입니다
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : # 앗 이경우에는 전값이 더 크네요? 그 경우에는 날짜가 바뀐것입니다.
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
        # 이 경우에는 얼마만큼 노출되었는지를 고려해서 그만큼을 MEAN 을 시켜줍니다. 
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : # 뒤의 값이 더 크네요? 그 경우에는 상품이 바뀐것이죠.
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])

In [276]:
# 마지막값은 그냥 무조건 값이 있어야겟죠?
idx = len(X)-1
if X.loc[idx,'월일'] in time_list:
    X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])
else:
    X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)'])-1,'시청률 평균'])

In [277]:
for idx in range(len(X)-1,-1,-1): # index 를 reverse 로 훑어들어오는것입니다. 
    if X.loc[idx,'시청률평균'] == 1: # 1이란것은 뒤의것과 겹친다는 이야기입니다.
        X.loc[idx,'시청률평균'] = X.loc[idx+1,'시청률평균'] 

In [278]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,2020-06-01,2020,06,01,6.333333,06:20,0601,620,20,0.001522
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,2020-06-01,2020,06,01,6.666667,06:40,0601,640,20,0.001564
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,2020-06-01,2020,06,01,7.000000,07:00,0601,700,20,0.001550
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,2020-06-01,2020,06,01,7.333333,07:20,0601,720,20,0.002319
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,2020-06-01,2020,06,01,7.666667,07:40,0601,740,20,0.003285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10,2020-07-01,2020,07,01,0.166667,00:10,0701,2410,10,0.005167
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20,2020-07-01,2020,07,01,1.333333,01:20,0701,2520,20,0.004828


In [279]:
X.to_excel('야호2!.xlsx')

## 시간 변수 세분화

특정 시간대에 홈쇼핑 방송 시청이 활발하거나, 홈쇼핑 상품 매출이 높을 것이라 생각하여 2시간으로 세분화한 시간 변수를 사용했다.

//식사시간의 경우 유의미한 데이터가 있을 수 있다. 그 시간대를 잡아서 변수를 늘리면 효과가 있을거같다.
하지만 아침 식사시간의 경우 거르는 인구가 33%(통계청) 이나 되는 이유로, 오히려 혼란을 가중시킬 수 있기떄문에 그냥 점심과 저녁만 고려하자

In [280]:
hour = X.loc[X['시간열']>2400,'시분']
hour = hour[~hour.duplicated()]
hour = hour.tolist()
X['시간열'].value_counts()

2020    78
1320    70
2040    70
2100    70
2220    64
        ..
805      1
2505     1
2435     1
2405     1
2335     1
Name: 시간열, Length: 100, dtype: int64

In [281]:
def time_2h(df = X):
    df['06:00~08:00'] = 0
    df['08:00~10:00'] = 0
    df['10:00~12:00'] = 0
    df['12:00~14:00'] = 0
    df['14:00~16:00'] = 0
    df['16:00~18:00'] = 0
    df['18:00~20:00'] = 0
    df['20:00~22:00'] = 0
    df['22:00~00:00'] = 0
    df['00:00~02:00'] = 0
    df['02:00~04:00'] = 0
     
    df.loc[(600<=df['시간열'])&(df['시간열']<800),'06:00~08:00'] = 1
    df.loc[(800<=df['시간열'])&(df['시간열']<1000),'08:00~10:00'] = 1
    df.loc[(1000<=df['시간열'])&(df['시간열']<1200),'10:00~12:00'] = 1
    df.loc[(1200<=df['시간열'])&(df['시간열']<1400),'12:00~14:00'] = 1
    df.loc[(1400<=df['시간열'])&(df['시간열']<1600),'14:00~16:00'] = 1
    df.loc[(1600<=df['시간열'])&(df['시간열']<1800),'16:00~18:00'] = 1
    df.loc[(1800<=df['시간열'])&(df['시간열']<2000),'18:00~20:00'] = 1
    df.loc[(2000<=df['시간열'])&(df['시간열']<2200),'20:00~22:00'] = 1
    df.loc[(2200<=df['시간열'])&(df['시간열']<2400),'22:00~00:00'] = 1
    df.loc[(2400<=df['시간열'])&(df['시간열']<2600),'00:00~02:00'] = 1
    df.loc[(2600<=df['시간열'])&(df['시간열']<2800),'02:00~04:00'] = 1
    return(df)

In [282]:
X = time_2h(X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,08:00~10:00,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0,0,0,0,0,0,0
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0,0,0,0,0,0,0
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0,0,0,0,0,0,0
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0,0,0,0,0,0,0
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2020-07-01 00:10:00,10.000000,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,6,수00:10,...,0,0,0,0,0,0,0,0,1,0
2712,2020-07-01 00:10:00,10.000000,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,6,수00:10,...,0,0,0,0,0,0,0,0,1,0
2713,2020-07-01 00:10:00,10.000000,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,6,수00:10,...,0,0,0,0,0,0,0,0,1,0
2714,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,20.000000,1,수01:20,...,0,0,0,0,0,0,0,0,1,0


## 강수량 변수

날씨 데이터의 경우 기상청의 공공데이터를 사용했다.
일별 기온(평균기온, 최고기온, 최저기온), 일별 강수량, 월별 기상정보(폭염일수, 열대야일수)를 고려했다.

In [283]:
rain_df=pd.read_csv(directory+'/20rain.csv',header = 6, encoding='CP949')
rain = rain_df.copy()

In [284]:
rain.head()

,날짜,지점,강수량(mm)
0,2020-06-01,전국,0.3
1,2020-06-02,전국,0.5
2,2020-06-03,전국,0.2
3,2020-06-04,전국,0.0
4,2020-06-05,전국,0.0


In [285]:
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,강수량(mm)
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0,0,0,0,0,0,0.3
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0,0,0,0,0,0,0.3
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0,0,0,0,0,0,0.3
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0,0,0,0,0,0,0.3
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0,0,0,0,0,0,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,0,0,0,0,0,1,0,0,27.9
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,0,0,0,0,0,1,0,0,27.9
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,0,0,0,0,0,1,0,0,27.9
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,0,0,0,0,0,1,0,0,27.9


## 온도 변수

In [286]:
temp_df=pd.read_csv(directory+'/20temp.csv',header = 6, encoding='CP949')
temp = temp_df.copy()

In [287]:
temp.head()

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2020-06-01,전국,20.8,15.8,26.0
1,2020-06-02,전국,19.5,13.4,25.4
2,2020-06-03,전국,22.0,16.9,27.5
3,2020-06-04,전국,24.0,18.8,29.9
4,2020-06-05,전국,23.1,18.6,29.3


In [288]:
def temp_feature(df = X):
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)
X = temp_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0,0,0,0.3,20.8,15.8,26.0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0,0,0,0.3,20.8,15.8,26.0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0,0,0,0.3,20.8,15.8,26.0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0,0,0,0.3,20.8,15.8,26.0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0,0,0,0.3,20.8,15.8,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,0,0,1,0,0,27.9,20.8,18.5,23.8
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,0,0,1,0,0,27.9,20.8,18.5,23.8
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,0,0,1,0,0,27.9,20.8,18.5,23.8
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,0,0,1,0,0,27.9,20.8,18.5,23.8


## 연예인 변수 

In [289]:
def celebrity(df=X):
    df['연예인']=0
    list_celebrity=["전지현", "팽현숙", "전철우", "강레오", "김선영", "김정배",
                    "김규흔", "이봉원", "오세득", "유귀열", "이경제", "이만기",
                    "이보은", "임성근", "전철우", "최인선", "김병만", "김병지", 
                    "이동수", "서장훈", "송도순", "효재", "천수봉", 
                    "김정문", "숀리", "이정섭", "황성주","이숙","임화자"]
    for i in list_celebrity:
        df.loc[df.상품명.str.contains(i),'연예인']=1
    return df

In [290]:
X=celebrity(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0,0,0.3,20.8,15.8,26.0,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0,0,0.3,20.8,15.8,26.0,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0,0,0.3,20.8,15.8,26.0,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0,0,0.3,20.8,15.8,26.0,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0,0,0.3,20.8,15.8,26.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,0,1,0,0,27.9,20.8,18.5,23.8,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,0,1,0,0,27.9,20.8,18.5,23.8,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,0,1,0,0,27.9,20.8,18.5,23.8,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,0,1,0,0,27.9,20.8,18.5,23.8,0


In [291]:
print(X.sum().연예인) #연예인 이름을 사용한 총 데이터 수

89


## 월급날

In [292]:
#https://news.sbs.co.kr/news/endPage.do?news_id=N1005505452
#https://m.blog.naver.com/youngkbblog/221714986107

In [293]:
def income(df = X):
    df['월급날'] = 0
    df.loc[ ((10<=df['일'].astype('float32'))&(df['일'].astype('float32')<=13)) | ((20<=df['일'].astype('float32'))&(df['일'].astype('float32')<= 23)) | ((25<=df['일'].astype('float32')) & (df['일'].astype('float32')<= 28)),'월급날']  = 1
    return(df)

In [294]:
X=income(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0,0.3,20.8,15.8,26.0,0,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0,0.3,20.8,15.8,26.0,0,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0,0.3,20.8,15.8,26.0,0,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0,0.3,20.8,15.8,26.0,0,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0,0.3,20.8,15.8,26.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,1,0,0,27.9,20.8,18.5,23.8,0,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,1,0,0,27.9,20.8,18.5,23.8,0,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,1,0,0,27.9,20.8,18.5,23.8,0,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,1,0,0,27.9,20.8,18.5,23.8,0,0


In [295]:
X.loc[X["일"]=="10", ["월급날"]] #잘 들어갔는지 확인
print(X.sum().월급날) #총 월급날

1093


## 금,토 및 공휴일

In [296]:
#https://0muwon.com/entry/2019%EB%85%84-%ED%9C%B4%EC%9D%BC-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%EB%8C%80%EC%B2%B4%ED%9C%B4%EC%9D%BC-%ED%99%A9%EA%B8%88%EC%97%B0%ED%9C%B4-%EC%9B%94%EB%B3%84-%EC%99%84%EB%B2%BD%EC%A0%95%EB%A6%AC

In [297]:
def holiday(df = X):
    df["휴일"] = 0
    holiday_list=[5,6,7,12,13,14,19,20,21,26,27,28]
    
    for i in holiday_list:
            df.loc[df['일'].astype('float32')==i,'휴일']=1
    return X

In [298]:
X=holiday(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,22:00~00:00,00:00~02:00,02:00~04:00,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,0.3,20.8,15.8,26.0,0,0,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,0.3,20.8,15.8,26.0,0,0,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,0.3,20.8,15.8,26.0,0,0,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,0.3,20.8,15.8,26.0,0,0,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,0.3,20.8,15.8,26.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,1,0,0,27.9,20.8,18.5,23.8,0,0,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,1,0,0,27.9,20.8,18.5,23.8,0,0,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,1,0,0,27.9,20.8,18.5,23.8,0,0,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,1,0,0,27.9,20.8,18.5,23.8,0,0,0


In [299]:
X.loc[(X["월"].astype("float32")==3) & (X["일"].astype("float32")==1),["휴일"]] #잘 들어갔는지 확인
print(X.sum().휴일) #총 휴일

1209


## 상품군 power

In [300]:
def goods_power(df=X):
    df.loc[:,"상품군power"] = 0
    goods = ['가구','가전','건강기능','농수축','생활용품','속옷','의류','이미용','잡화','주방','침구']
    power = [3.85, 2.95, 22.41, 52.04, 16.92, 24.38, 26.38, 41.32, 25.8, 19.35, 25.03]
    for i in range(len(df)):
        for j in range(len(goods)):
            if df.loc[i,'상품군'] == goods[j]:
                df.loc[i,"상품군power"] = power[j]
    return(df)

In [301]:
len(X)

2708

In [302]:
X = goods_power(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,00:00~02:00,02:00~04:00,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0.3,20.8,15.8,26.0,0,0,0,26.38
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0.3,20.8,15.8,26.0,0,0,0,26.38
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0.3,20.8,15.8,26.0,0,0,0,26.38
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0.3,20.8,15.8,26.0,0,0,0,24.38
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0.3,20.8,15.8,26.0,0,0,0,24.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,0,27.9,20.8,18.5,23.8,0,0,0,24.38
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,0,27.9,20.8,18.5,23.8,0,0,0,24.38
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,0,27.9,20.8,18.5,23.8,0,0,0,24.38
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,0,27.9,20.8,18.5,23.8,0,0,0,24.38


In [303]:
#X.to_excel('테스트중중.xlsx')

## 소비자물가지수

In [304]:
X['CPI'] = 0

## 소비자 동향지수

In [305]:
X['CSI'] = 81.8

## 소매업태별(홈쇼핑) 판매액지수(불변지수)

In [306]:
X['RS'] = 139.2

## 시청률 data

주파수를 사용하지 않는 케이블, 위성방송, IPTV의 등장에 따라 셋톱박스에서 조사가 가정해져서 시청률이 좀 더 정확해졌으며, 시청률 조사지역은 점차 확대되어 수도권, 5대 광역시, 중소도시등이 포함되어 2019년에는 모든 케이블 방송권역에서 조사가 이루어지고 있다. 2020년 현재 TNMS는 전국(제주도 및 읍·면 지역 포함) 4,000여 가구에 거주하는 약 9천명을 셋탑박스를 조사(TMNS 홈페이지 기준), 닐슨 코리아에서는 피플미터기로 4,220가구(닐슨 컴퍼니 코리아 홈페이지 기준)를 조사한다.

+) http://www.mediaus.co.kr/news/articleView.html?idxno=22809

## 심야 시간대 표현

00 시 이후는 24를 더해주었습니다.

In [307]:
X.loc[X.loc[:,'시각'] < 3,'시각'] = X.loc[X.loc[:,'시각'] < 3,'시각'] + 24

In [308]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,20.8,15.8,26.0,0,0,0,26.38,0,81.8,139.2
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,20.8,15.8,26.0,0,0,0,26.38,0,81.8,139.2
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,20.8,15.8,26.0,0,0,0,26.38,0,81.8,139.2
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,20.8,15.8,26.0,0,0,0,24.38,0,81.8,139.2
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,20.8,15.8,26.0,0,0,0,24.38,0,81.8,139.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,20.8,18.5,23.8,0,0,0,24.38,0,81.8,139.2
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,20.8,18.5,23.8,0,0,0,24.38,0,81.8,139.2
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,20.8,18.5,23.8,0,0,0,24.38,0,81.8,139.2
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,20.8,18.5,23.8,0,0,0,24.38,0,81.8,139.2


In [309]:
X.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '평균방송분', '겹치는수',
       '요일/시간', '날짜', '년', '월', '일', '시각', '시분', '월일', '시간열', '정수노출(분)',
       '시청률평균', '06:00~08:00', '08:00~10:00', '10:00~12:00', '12:00~14:00',
       '14:00~16:00', '16:00~18:00', '18:00~20:00', '20:00~22:00',
       '22:00~00:00', '00:00~02:00', '02:00~04:00', '강수량(mm)', '평균기온(℃)',
       '최저기온(℃)', '최고기온(℃)', '연예인', '월급날', '휴일', '상품군power', 'CPI', 'CSI',
       'RS'],
      dtype='object')

## Prime Time

In [310]:
# https://www.etnews.com/20180718000049?m=1

In [311]:
## one-hot vector(프라임타임 or not)

def prime_time(df = X):
    df['PrimeTime'] = 0
    
    for i in range(len(df)):
        if (9 <= df.loc[i,'시각']) and (df.loc[i,'시각'] < 11):
            df.loc[i,'PrimeTime'] = 1
        elif df.loc[i,'상품군']=="농수축" and (16<=df.loc[i,'시각']) and (df.loc[i,'시각']< 18):
            df.loc[i,'PrimeTime'] = 1            
        elif df.loc[i,'상품군']!="농수축" and (20<=df.loc[i,'시각']) and (df.loc[i,'시각']< 23):
            df.loc[i,'PrimeTime'] = 1
    
    return df

In [312]:
prime_time(X) ;X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS,PrimeTime
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,15.8,26.0,0,0,0,26.38,0,81.8,139.2,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,15.8,26.0,0,0,0,26.38,0,81.8,139.2,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,15.8,26.0,0,0,0,26.38,0,81.8,139.2,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,15.8,26.0,0,0,0,24.38,0,81.8,139.2,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,15.8,26.0,0,0,0,24.38,0,81.8,139.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,18.5,23.8,0,0,0,24.38,0,81.8,139.2,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,18.5,23.8,0,0,0,24.38,0,81.8,139.2,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,18.5,23.8,0,0,0,24.38,0,81.8,139.2,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,18.5,23.8,0,0,0,24.38,0,81.8,139.2,0


In [313]:
X['PrimeTime'].sum()
## 생각보다 해당되는 항목이 많네요..!!
## 시간 처리할 때 ( , ) 이 아니라 [ , ) 로 했습니다.

944

## 남성, 여성 상품

In [314]:
def gender(df=X):
    df["남성상품"]=0
    df["여성상품"]=0
    list_male=["남성","드로즈","푸마 드라이셀라인 뉴 런닝","언더셔츠","카파 런닝","리복 스피드윅 런닝",
               "트렁크","프로스펙스 2020 퍼포먼스 런닝"]
    list_female=["여성","브라","밍크","팬티","치마","레깅스","란쥬","핸드백",
                 "앙상블","엔셀라두스 밴딩팬츠","레이스","엘리자베스아덴","버켓 백",
                 "크로스바디 백","립스틱","쿠션","사첼백","숄더","속눈썹","토트백",
                 "원피스","틴트","투웨이백","이지탑","크로스백","클러치백","보정런닝",
                 "오렐리안 히트모 스킨컬렉션 기모런닝","보정 런닝"]
    for i in ["의류","속옷","잡화","이미용"]:
        for j in list_male:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(j)),"남성상품"]=1
        for k in list_female:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(k)),"여성상품"]=1
    return df

In [315]:
X = gender(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,연예인,월급날,휴일,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,0,0,26.38,0,81.8,139.2,0,1,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,0,0,26.38,0,81.8,139.2,0,1,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,0,0,26.38,0,81.8,139.2,0,1,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,0,0,24.38,0,81.8,139.2,0,0,1
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,0,0,24.38,0,81.8,139.2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,0,0,24.38,0,81.8,139.2,0,1,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,0,0,24.38,0,81.8,139.2,0,1,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,0,0,24.38,0,81.8,139.2,0,1,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,0,0,24.38,0,81.8,139.2,0,1,0


## 무이자, 일시불 처리

무이자, 일시불이 소비에 영향을 미칠 것이라 판단하여 상품 별 무이자, 일시불을 나타내는 열 삽입

In [316]:
def pay(df=X):
    df["무이자"]=0
    df["일시불"]=0
    pay_in_inst=["무)", "무이자"]
    pay_in_full=["일)", "일시불"]
    for i in pay_in_inst:
        df.loc[(df.상품명.str.contains(i,regex=False))&(~df.상품명.str.contains("보험")),"무이자"]=1
    for j in pay_in_full:
        df.loc[(df.상품명.str.contains(j, regex=False))&(~df.상품명.str.contains("세일")),"일시불"]=1
    return df

In [317]:
X = pay(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,휴일,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,26.38,0,81.8,139.2,0,1,0,0,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,26.38,0,81.8,139.2,0,1,0,0,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,26.38,0,81.8,139.2,0,1,0,0,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,24.38,0,81.8,139.2,0,0,1,0,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,24.38,0,81.8,139.2,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,24.38,0,81.8,139.2,0,1,0,1,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,24.38,0,81.8,139.2,0,1,0,0,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,24.38,0,81.8,139.2,0,1,0,0,1
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,24.38,0,81.8,139.2,0,1,0,1,0


In [318]:
# 확인용
X.loc[X['무이자']==1, '상품명']

135                     무이자 LG전자 매직스페이스 냉장고
137                     무이자 LG전자 매직스페이스 냉장고
139                     무이자 LG전자 매직스페이스 냉장고
140      무이자 삼성 UHDTV 75형  / KU75UT7000FXKR
142         무이자 삼성 UHDTV 65형 KU65UT7000FXKR
                       ...                 
2572    무이자 올리고 가스와이드그릴레인지 프리미엄형 +버팔로 캠핑쿨러백
2697               [무이자]라쉬반 FC바로셀로나 드로즈 패키지
2700               [무이자]라쉬반 FC바로셀로나 드로즈 패키지
2703               [무이자]라쉬반 FC바로셀로나 드로즈 패키지
2706               [무이자]라쉬반 FC바로셀로나 드로즈 패키지
Name: 상품명, Length: 226, dtype: object

## 유명 기업/브랜드명

In [319]:
def brandname(df=X):
    df['유명기업/브랜드']=0
    lst=["LG","SPC삼립","경남제약", "광동", "구찌", "굿프렌드", "농협", "딤채",
         "락앤락","리복","마리끌레르","뱅뱅","베지밀","보루네오","본죽","삼성",
         "엘르","종근당건강","쿠쿠",'쿠첸',"푸마","프라다","하림","한샘",'버버리',
         '비비고','생로랑']
    for i in lst:
        df.loc[df.상품명.str.contains(i),'유명기업/브랜드']=1
    for i in range(len(df)):
        if ("폴로" in df.loc[i,"상품명"]) or ("USPA" in df.loc[i,"상품명"]):
            df.loc[i,"유명기업/브랜드"] = 1
            
    return df

In [320]:
X = brandname(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,26.38,0,81.8,139.2,0,1,0,0,0,0
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,26.38,0,81.8,139.2,0,1,0,0,0,0
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,26.38,0,81.8,139.2,0,1,0,0,0,0
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,24.38,0,81.8,139.2,0,0,1,0,0,0
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,24.38,0,81.8,139.2,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,24.38,0,81.8,139.2,0,1,0,1,0,0
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,24.38,0,81.8,139.2,0,1,0,0,0,0
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,24.38,0,81.8,139.2,0,1,0,0,1,0
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,24.38,0,81.8,139.2,0,1,0,1,0,0


# 타 채널 시청자 수

In [321]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [322]:
date = X['월일']
date = date[~date.duplicated()]
date = list(date)

In [323]:
X['타 채널 시청자 수 평균'] = 0
def rate_add(df=X, date=date):
    
    for i in date:
        url = "http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date=2020"+i
        html = urlopen(url)  
        rate_html = BeautifulSoup(html, "html.parser") 
        rate = []
        for j in range(10,20):
            a = rate_html.find_all("td",class_='percent')[j].get_text()
            a=a.replace("\t", "")
            a=a.replace("\r","")
            a=a.replace("\n","")
            a=a.replace(",","")
            a=a.strip()
            rate.append(a)
            
        for k in range(10,20):
            b = rate_html.find_all("td",class_='percent_g')[k].get_text()
            b=b.replace("\t", "")
            b=b.replace("\r","")
            b=b.replace("\n","")
            b=b.replace(",","")
            b=b.strip()
            rate.append(b)
           
        rate = [int(m) for m in rate]
        rate_mean = sum(rate) / 20
        df.loc[df['월일']==i,'타 채널 시청자 수 평균'] = rate_mean
           
    return  df

In [324]:
X = rate_add(X, date)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,0,81.8,139.2,0,1,0,0,0,0,1382.15
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,0,81.8,139.2,0,1,0,0,0,0,1382.15
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,0,81.8,139.2,0,1,0,0,0,0,1382.15
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,0,81.8,139.2,0,0,1,0,0,0,1382.15
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,0,81.8,139.2,0,0,1,0,0,0,1382.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,0,81.8,139.2,0,1,0,1,0,0,1307.80
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,0,81.8,139.2,0,1,0,0,0,0,1307.80
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,0,81.8,139.2,0,1,0,0,1,0,1307.80
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,0,81.8,139.2,0,1,0,1,0,0,1307.80


## 가전제품 소분류 추가

In [325]:
X["가전제품"]="가전제품 아님"
X["가전제품"]

0       가전제품 아님
1       가전제품 아님
2       가전제품 아님
3       가전제품 아님
4       가전제품 아님
         ...   
2703    가전제품 아님
2704    가전제품 아님
2705    가전제품 아님
2706    가전제품 아님
2707    가전제품 아님
Name: 가전제품, Length: 2708, dtype: object

In [326]:
list_products=['세탁기', '노트북', '냉장고', 'TV', '건조기', '미니건조기', '공기청정기', '무선청소기', '에어컨', '의류관리기']

for j in list_products:
    for i in range(len(X)):
        if X.loc[i,"상품군"]=="가전":
            if j in X.loc[i,"상품명"]:
                X.loc[i,"가전제품"]=j
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,81.8,139.2,0,1,0,0,0,0,1382.15,가전제품 아님
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,81.8,139.2,0,1,0,0,0,0,1382.15,가전제품 아님
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,81.8,139.2,0,1,0,0,0,0,1382.15,가전제품 아님
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,81.8,139.2,0,0,1,0,0,0,1382.15,가전제품 아님
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,81.8,139.2,0,0,1,0,0,0,1382.15,가전제품 아님
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,81.8,139.2,0,1,0,1,0,0,1307.80,가전제품 아님
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,81.8,139.2,0,1,0,0,0,0,1307.80,가전제품 아님
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,81.8,139.2,0,1,0,0,1,0,1307.80,가전제품 아님
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,81.8,139.2,0,1,0,1,0,0,1307.80,가전제품 아님


# 농수축소분류

In [327]:
def AgroFishery(df=X):
    df['농수축소분류']="농수축아님"
    lst_쌀 = ["서산뜸부기쌀","멋진밥상 흥양농협 햅쌀"]
    lst_과일 = ["키위","감귤"]
    lst_김치 = ["김치"]
    lst_어류 = ["제주바다자연산돔","오징어","우럭","가자미","전복","고등어","굴비","홍어","[오세득 프라임 컬렉션] 바로 메로 10팩+특제소스","갈치","꽃게","장어","문어"]
    lst_건어류 = ["쥐포","쥐치포"]
    lst_육류 = ["닭발","뒷고기","곱창","오리","육포","삼계탕","해장국"]
    lst_음료 = ["커피","아침애끌리다","두유"]
    lst_주전부리 = ["오메기떡", "전철우냉면", "다시마국수","만두","갓바위대천김도시락김120봉","갓바위전장김27봉"]
    lst_채소 = ["강원도산골더덕무침10팩"]
    
    for i in lst_쌀:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="쌀"
    for i in lst_과일:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="과일"
    for i in lst_김치:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="김치"
    for i in lst_어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="어류"
    for i in lst_건어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="건어류"
    for i in lst_육류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="육류"
    for i in lst_음료:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="음료"
    for i in lst_주전부리:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="주전부리"
    for i in lst_채소:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="채소"


    return df

In [328]:
X = AgroFishery(X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,평균방송분,겹치는수,요일/시간,...,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품,농수축소분류
0,2020-06-01 06:20:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,1,월06:20,...,139.2,0,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
1,2020-06-01 06:40:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,2,월06:40,...,139.2,0,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
2,2020-06-01 07:00:00,20.0,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,20.000000,3,월07:00,...,139.2,0,1,0,0,0,0,1382.15,가전제품 아님,농수축아님
3,2020-06-01 07:20:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,1,월07:20,...,139.2,0,0,1,0,0,0,1382.15,가전제품 아님,농수축아님
4,2020-06-01 07:40:00,20.0,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20.000000,2,월07:40,...,139.2,0,0,1,0,0,0,1382.15,가전제품 아님,농수축아님
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2020-06-30 23:40:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,3,화23:40,...,139.2,0,1,0,1,0,0,1307.80,가전제품 아님,농수축아님
2704,2020-06-30 23:40:00,10.0,100099,200274,라쉬반 FC바로셀로나 드로즈 8종,속옷,119000,3.333333,3,화23:40,...,139.2,0,1,0,0,0,0,1307.80,가전제품 아님,농수축아님
2705,2020-06-30 23:50:00,10.0,100099,200273,[일시불]라쉬반 FC바로셀로나 드로즈 패키지,속옷,99000,3.333333,4,화23:50,...,139.2,0,1,0,0,1,0,1307.80,가전제품 아님,농수축아님
2706,2020-06-30 23:50:00,10.0,100099,200272,[무이자]라쉬반 FC바로셀로나 드로즈 패키지,속옷,119000,3.333333,4,화23:50,...,139.2,0,1,0,1,0,0,1307.80,가전제품 아님,농수축아님


# 필요없는 변수삭제

In [329]:
X.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '평균방송분', '겹치는수',
       '요일/시간', '날짜', '년', '월', '일', '시각', '시분', '월일', '시간열', '정수노출(분)',
       '시청률평균', '06:00~08:00', '08:00~10:00', '10:00~12:00', '12:00~14:00',
       '14:00~16:00', '16:00~18:00', '18:00~20:00', '20:00~22:00',
       '22:00~00:00', '00:00~02:00', '02:00~04:00', '강수량(mm)', '평균기온(℃)',
       '최저기온(℃)', '최고기온(℃)', '연예인', '월급날', '휴일', '상품군power', 'CPI', 'CSI',
       'RS', 'PrimeTime', '남성상품', '여성상품', '무이자', '일시불', '유명기업/브랜드',
       '타 채널 시청자 수 평균', '가전제품', '농수축소분류'],
      dtype='object')

In [330]:
X.loc[:,'상품군power'].unique()

array([26.38, 24.38, 41.32, 19.35, 52.04,  2.95, 16.92,  3.85, 25.8 ,
       22.41, 25.03])

In [331]:
#X.drop(columns=['방송일시','마더코드','상품코드','상품명','요일/시간','날짜','년','월','일','시분','상품군power','가전제품'],inplace=True)

In [332]:
#X

In [333]:
X.to_excel('processed.xlsx')